### Producers and Consumers with Coroutines (Python) [4 points]

Here is a solution to the producer-consumer problem with thread-safe queues in Python. There are each `numProdsCons` producers and consumers. Each producer places the first `numIters` numbers in a shared buffer. Each consumer takes `numIters` numbers from the shared buffer and sends their sum to a queue with the results. The main program prints the sum of all the results as well as the expected total sum:

In [1]:
from threading import Thread
from queue import Queue

def producer(n: int, b: Queue):
    for i in range(n): b.put(i)

def consumer(n: int, b: Queue, r: Queue):
    s = 0
    for _ in range(n): s += b.get()
    r.put(s)

def pc(numIters: int, numProdsCons: int, capacity: int): # capacity = 0 for unbounded queue
    buf, result = Queue(capacity), Queue(numProdsCons)
    for _ in range(numProdsCons): Thread(target = producer, args = (numIters, buf)).start()
    for _ in range(numProdsCons): Thread(target = consumer, args = (numIters, buf, result)).start()
    s = 0
    for _ in range(numProdsCons): s += result.get()
    print("The computed total sum is", s)
    print("The expected total sum is", numIters * (numIters - 1) // 2 * numProdsCons)

In [2]:
from time import perf_counter
start = perf_counter()
pc(1000, 50, 10)
end = perf_counter()
print("Elapsed time", round(end - start, 3), "sec")

The computed total sum is 24975000
The expected total sum is 24975000
Elapsed time 0.55 sec


Your task is to re-implement the above solution with coroutines instead of threads. That is, each producer, each consumer, and the main function `pc` should be coroutines and communicate `Queue` objects of the `asyncio` library. The output should be identical to the threading version:

In [16]:
from asyncio import create_task, Queue

async def producer(n: int, b: Queue):
    for i in range(n): await b.put(i)

async def consumer(n: int, b: Queue, r: Queue):
    s = 0
    for _ in range(n): 
        item = await b.get()
        s += item
    await r.put(s)

async def pc(numIters: int, numProdsCons: int, capacity: int): 
    buf, result = Queue(capacity), Queue(numProdsCons)
    producer_tasks = [create_task(producer(numIters, buf)) for _ in range(numProdsCons)]
    consumer_tasks = [create_task(consumer(numIters, buf, result)) for _ in range(numProdsCons)]

    for p in producer_tasks: await p
    for c in consumer_tasks: await c
    
    s = 0
    for _ in range(numProdsCons): 
        item = await result.get()
        s += item
        
    print("The computed total sum is", s)
    print("The expected total sum is", numIters * (numIters - 1) // 2 * numProdsCons)

In [17]:
from time import perf_counter
start = perf_counter()
await pc(1000, 50, 0) # replace with pc(1000, 50, 10) if running in own thread
end = perf_counter()
print("Elapsed time", round(end - start, 3), "sec")

The computed total sum is 24975000
The expected total sum is 24975000
Elapsed time 0.051 sec


1. What do you observe when increasing `numProdsCons` and `capacity` in the threading version? What is your explanation?
2. What do you observe when increasing `numProdsCons` and `capacity` in the coroutine version? What is your explanation?
3. What are your observations when comparing the threading and the coroutine versions? What are your explanations?

YOUR ANSWER HERE